# Demo: Fine-tuning with Geometry Image Modality for Regression Task

In [1]:
import numpy as np
from torch import nn
from torch.optim import Adam
from benchmol.dataloader.image_dataset import TrainValTestFromCSVFactory
from benchmol.trainer import Trainer
from benchmol.model_pools import ImageModelFactory

In [2]:
# Data
dataroot = "../datasets/toys/CHEMBL4419606_IC50_nM/processed"
csv_path = "../datasets/toys/CHEMBL4419606_IC50_nM/processed/CHEMBL4419606_IC50_nM_processed_ac.csv"
data_type = "multi_view_image"  # geometry image modality
num_tasks = 1
device = "cuda:1"  # cpu or cuda:0

# Take the image modality and regression task as an example
modality="image"
task_type = "regression"

# define Model with backbone as ViT and neck as arch4 for n tasks
model = ImageModelFactory(
    model_name="vit_small_patch16_224", head_arch="arch4",
    head_arch_params={"inner_dim": 128, "dropout": 0.2, "activation_fn": "gelu"}, num_tasks=num_tasks
)
# define Dataset
factory = TrainValTestFromCSVFactory(
    dataroot, csv_path, data_type=data_type, image_dir_name="rdkit/type-IEM", task_type=task_type,
    batch_size=8, num_workers=8
)
train_loader, valid_loader, test_loader = factory.get_dataloader(split="train"), factory.get_dataloader(split="valid"), factory.get_dataloader(split="test")

# define Trainer
trainer = Trainer(
model, modality, train_loader, valid_loader, test_loader, task_type,
    criterion=nn.MSELoss(), optimizer=Adam(model.parameters(), lr=0.005, weight_decay=1e-5), device=device
)

# training and evaluation
results = trainer.train(
    num_epochs=20, eval_metric="RMSE", valid_select="min", min_value=np.inf, dataset="CHEMBL4419606_IC50_nM", 
    save_finetune_ckpt=True, save_dir="./experiments/"
)

# Output model results
print("results: {}\n".format(results))

[split=train] use transform
[train] split_image_path_list[0]: ['../datasets/toys/CHEMBL4419606_IC50_nM/processed/rdkit/type-IEM/9886677/x_0.png', '../datasets/toys/CHEMBL4419606_IC50_nM/processed/rdkit/type-IEM/9886677/x_180.png', '../datasets/toys/CHEMBL4419606_IC50_nM/processed/rdkit/type-IEM/9886677/y_180.png', '../datasets/toys/CHEMBL4419606_IC50_nM/processed/rdkit/type-IEM/9886677/z_180.png']
[split=valid] use eval_transform
[split=test] use eval_transform


[train] dataset: CHEMBL4419606_IC50_nM; epoch: 0 total loss: 3.787: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:05<00:00,  2.85it/s]
[eval on train set] dataset: CHEMBL4419606_IC50_nM; epoch: 0 total loss: 0.770: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:01<00:00, 12.21it/s]
[eval on valid set] dataset: CHEMBL4419606_IC50_nM; epoch: 0 total loss: 1.050: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  5.10it/s

{'dataset': 'CHEMBL4419606_IC50_nM', 'epoch': 0, 'Loss': 0.7697847485542297, 'metric': 'RMSE', 'train': 0.8792040557064762, 'valid': 1.0602266763944883, 'test': 1.063571288395671}
model has been saved as ./experiments//valid_best.pth


[train] dataset: CHEMBL4419606_IC50_nM; epoch: 1 total loss: 0.553: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:01<00:00,  8.01it/s]
[eval on train set] dataset: CHEMBL4419606_IC50_nM; epoch: 1 total loss: 0.376: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:01<00:00, 12.00it/s]
[eval on valid set] dataset: CHEMBL4419606_IC50_nM; epoch: 1 total loss: 1.494: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  5.51it/s

{'dataset': 'CHEMBL4419606_IC50_nM', 'epoch': 1, 'Loss': 0.37556466460227966, 'metric': 'RMSE', 'train': 0.6161254860797364, 'valid': 1.1758282408165055, 'test': 0.879728621559283}


...

{'dataset': 'CHEMBL4419606_IC50_nM', 'epoch': 17, 'Loss': 0.3797190487384796, 'metric': 'RMSE', 'train': 0.616689194633462, 'valid': 1.1840612088337412, 'test': 0.8822505672635323}


[train] dataset: CHEMBL4419606_IC50_nM; epoch: 18 total loss: 0.377: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:02<00:00,  7.53it/s]
[eval on train set] dataset: CHEMBL4419606_IC50_nM; epoch: 18 total loss: 0.560: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:01<00:00, 12.04it/s]
[eval on valid set] dataset: CHEMBL4419606_IC50_nM; epoch: 18 total loss: 2.189: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  5.37it/s

{'dataset': 'CHEMBL4419606_IC50_nM', 'epoch': 18, 'Loss': 0.5598130822181702, 'metric': 'RMSE', 'train': 0.7566296638689841, 'valid': 1.4048993329497714, 'test': 0.9964382748488649}


[train] dataset: CHEMBL4419606_IC50_nM; epoch: 19 total loss: 0.632: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:02<00:00,  7.83it/s]
[eval on train set] dataset: CHEMBL4419606_IC50_nM; epoch: 19 total loss: 0.416: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 16/16 [00:01<00:00, 11.80it/s]
[eval on valid set] dataset: CHEMBL4419606_IC50_nM; epoch: 19 total loss: 1.661: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  4.95it/s

{'dataset': 'CHEMBL4419606_IC50_nM', 'epoch': 19, 'Loss': 0.41610944271087646, 'metric': 'RMSE', 'train': 0.6335449965083038, 'valid': 1.2334857552395462, 'test': 0.8976160082894971}
results: {'highest_valid': 1.050263447367695, 'final_train': 0.8434753063501916, 'final_test': 1.0356775994320604, 'highest_train': 0.6161254860797364}

